# Agent Memory Project

## Load env

In [1]:
import os
import openai

from openai import OpenAI
from dotenv import load_dotenv
from pathlib import Path

# Load path from the environment variable
env_ih1 = os.getenv("ENV_IH1")

dotenv_path = Path(env_ih1)
load_dotenv(dotenv_path=dotenv_path)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY= os.getenv('PINECONE_KEY')
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')
STEAMSHIP_API_KEY = os.getenv('STEAMSHIP_API_KEY')
LANGSMITH_API_KEY = os.getenv('LANGSMITH_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')
GEMINI_KEY = os.getenv('GEMINI_KEY')

os.environ['PATH'] += os.pathsep + '/usr/bin'

In [2]:
from langsmith import wrappers, traceable

LANGSMITH_API_KEY= LANGSMITH_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="memory-project"

## Load Pinecone DB

In [28]:
import time
from pinecone import Pinecone

# configure client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Connect to the existing index
index_name = "memory-project4"  # Replace with your existing index name
# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 170}},
 'total_vector_count': 170}

In [4]:
from langchain.vectorstores import Pinecone as LangchainPinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import Document

## Create encoder
from semantic_router.encoders import OpenAIEncoder

encoder = OpenAIEncoder(
    name="text-embedding-3-small",
    openai_api_key=OPENAI_API_KEY 
)

## Creating retriever

# Initialize OpenAI Embeddings with text-embedding-3-small
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=OPENAI_API_KEY)

# Initialize LangChain Pinecone retriever
vectorstore = LangchainPinecone(index, embeddings, text_key="text")
retriever = vectorstore.as_retriever()

C:\Users\aurel\AppData\Local\Temp\ipykernel_23468\1506975490.py:16: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=OPENAI_API_KEY)
C:\Users\aurel\AppData\Local\Temp\ipykernel_23468\1506975490.py:19: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-pinecone package and should be used instead. To use it run `pip install -U :class:`~langchain-pinecone` and import as `from :class:`~langchain_pinecone import Pinecone``.
  vectorstore = LangchainPinecone(index, embeddings, text_key=

In [5]:
# Test retrieval
query = "Who is Jean Lambert?"
results = retriever.get_relevant_documents(query)

# Print results
for doc in results:
    print(f"Content: {doc.page_content}")
    print(f"Metadata: {doc.metadata}")
    print("-" * 50)

C:\Users\aurel\AppData\Local\Temp\ipykernel_23468\3093734283.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)


Content: POUR LA MÉMOIRE
FAMILIALE

FAMILLE HISTOIRE
SOUVENIRS & COMMENTAIRES

VOLUME 1

Jean-Georges Lambert
Avril 1993
Metadata: {'Author': 'Jean Lambert', 'Chunk_ID': 'Pour la mémoire familiale 1-50_Chunk1', 'Doc name': 'Pour la mémoire familiale 1-50', 'Page_number': 1.0, 'Total_Chunks': 1.0}
--------------------------------------------------
Content: de Maman Michel, et il est mort à Reims le 4 mars 1893 âgé de soixante quatre ans.

Son influence sur la famille est de plus courte durée que celle de Maman Michel, mais c'est aussi une forte personnalité qui a joué un rôle important dans le déroulement de la vie familiale.

L'histoire de Jacques Dreyfus commence à l'époque où il exerçait la profession de colporteur. C'est le terme qu'employait mon père, mais je pense qu'il n'est pas exact, en raison de l'importance de son commerce. Je dirais plutôt négociant en tissus, en précisant qu'il l'exerçait en ambulant. Il avait pour cela une voiture à deux chevaux assez spacieuse et conforta

## Create Graph Agent

In [6]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator


class AgentState(TypedDict):
    input: str
    chat_history: list[BaseMessage]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

## Create custom tools

In [7]:
from langchain_core.tools import tool
import re


def format_rag_contexts(matches: list):
    """Formats retrieved document matches into a readable context string."""
    contexts = []
    for x in matches:
        metadata = x.get("metadata", {})  # Safely get metadata
        text = (
            f"Doc name: {metadata.get('Doc name', 'N/A')}\n"
            f"Author: {metadata.get('Author', 'N/A')}\n"
            f"Chunk_ID: {metadata.get('Chunk_ID', 'N/A')}\n"
            f"Content: {metadata.get('text', 'No content available')}"  # Fixed page content reference
        )
        contexts.append(text)

    return "\n---\n".join(contexts)

""""
@tool("simple_search")  # Fixed invalid tool name
"""
# def simple_search(query: str):
#     """Finds related information using a natural language query in the family history."""
#     results = retriever.get_relevant_documents(query)
#     return format_rag_contexts(results)  # Return formatted results instead of raw objects

@tool("translate_to_french")
def translate_to_french(query: str):
    """Translates the given query to French using OpenAI's `o4-mini` model to improve retriebal in French database."""
    
    response = openai.ChatCompletion.create(
        model="gpt-4o",  # "o4-mini" may refer to "gpt-4o"
        messages=[
            {"role": "system", "content": "You are a translation assistant."},
            {"role": "user", "content": f"Translate the following text to French:\n{query}"}
        ],
        temperature=0
    )

    return response

    

from typing import List
from langchain.tools import tool
import re


@tool("rag_search")
def rag_search(query: str):
    """Finds related information using a natural language query in the family history."""

    xq = encoder([query])
    xc = index.query(vector=xq, top_k=5, include_metadata=True)

    if "matches" not in xc or not xc["matches"]:
        return "⚠ No relevant documents found."

    # Extract best match details
    best_match = xc["matches"][0]
    chunk_id = best_match["metadata"].get("Chunk_ID", "")
    doc_name = best_match["metadata"].get("Doc name", "")
    total_chunks = best_match["metadata"].get("Total_Chunks", 1)  # Default to 1 if missing

    try:
        total_chunks = int(float(total_chunks))  # Ensure integer conversion from float or string
    except (ValueError, TypeError):
        total_chunks = 1  # Fallback in case of conversion error

    print(f"🔍 Best Match - Doc: {doc_name}, Chunk: {chunk_id}, Total Chunks: {total_chunks}")

    # Extract chunk number
    match = re.match(r"(.+)_Chunk(\d+)", chunk_id)
    if not match:
        return format_rag_contexts(xc["matches"])  # Return base context if extraction fails

    _, chunk_num = match.groups()

    try:
        chunk_num = int(chunk_num)  # Ensure chunk_num is an integer
    except ValueError:
        return format_rag_contexts(xc["matches"])  # Fallback in case of error

    # Debug: Print extracted chunk numbers before range calculation
    print(f"🔢 Extracted Chunk Number: {chunk_num}, Total Chunks: {total_chunks}")

    # Generate **bounded** chunk IDs within valid range
    min_chunk = max(1, chunk_num - 3)
    max_chunk = min(total_chunks, chunk_num + 3)

    # Debug: Check type before using in range
    print(f"🔍 Before Casting - min_chunk: {min_chunk} ({type(min_chunk)}), max_chunk: {max_chunk} ({type(max_chunk)})")

    # Ensure integer conversion
    min_chunk = int(min_chunk)
    max_chunk = int(max_chunk)

    # Debug: Confirm type after casting
    print(f"✅ After Casting - min_chunk: {min_chunk} ({type(min_chunk)}), max_chunk: {max_chunk} ({type(max_chunk)})")

    expanded_chunk_ids = [f"{doc_name}_Chunk{i}" for i in range(min_chunk, max_chunk + 1)]

    print(f"🧩 Expanding context with bounded chunks: {expanded_chunk_ids}")

    # Retrieve expanded chunks
    expanded_contexts = []
    for expanded_chunk in expanded_chunk_ids:
        try:
            result = chunk_search.run({"query": query, "chunk_id": expanded_chunk})
            if result and "⚠ No context found" not in result:
                expanded_contexts.append(result)
        except Exception as e:
            print(f"⚠ Error retrieving chunk {expanded_chunk}: {str(e)}")

    # Combine and return formatted context
    full_context = "\n\n".join(expanded_contexts) if expanded_contexts else format_rag_contexts(xc["matches"])
    return full_context






@tool("chunk_search")
def chunk_search(query: str, chunk_id: str):
    """Finds related information based on the chunk_id. Helps to get more context."""

    # Extract Doc Name and Chunk Number from chunk_id
    match = re.match(r"(.+)_Chunk(\d+)", chunk_id)
    if not match:
        return f"❌ Error: Invalid chunk_id format '{chunk_id}'"

    doc_name, chunk_number = match.groups()
    chunk_number = int(chunk_number)

    # Generate surrounding chunk IDs (-2, -1, +1, +2)
    chunk_ids = [f"{doc_name}_Chunk{i}" for i in range(chunk_number - 8, chunk_number + 8) if i > 0]

    print(f"🔍 Debugging Chunk IDs: {chunk_ids}")

    # Encode query into a vector (Replace `encoder()` with your actual embedding function)
    xq = encoder([query])  # Convert query into vector

    try:
        # Perform a **vector-based search** instead of only filtering by chunk ID
        response = index.query(
            vector=xq,
            namespace="",
            filter={"Chunk_ID": {"$in": chunk_ids}},  # Ensure correct filter usage
            top_k=15,  # Fetch relevant results
            include_metadata=True
        )
        direct_matches = response["matches"]

    except Exception as e:
        return f"❌ Pinecone Query Error: {str(e)}"

    # Format and return results
    results = []
    for match in direct_matches:
        metadata = match.get("metadata", {})
        chunk_text = metadata.get("text", "No content available")
        chunk_id_match = metadata.get("Chunk_ID", "Unknown Chunk")
        results.append(f"📜 Found Chunk: {chunk_id_match} -> {chunk_text}...")  # Limit text preview

    return "\n".join(results) if results else "⚠ No relevant chunks found."


In [8]:
rag_search.run("29 Mars 1963")

🔍 Best Match - Doc: Pour la mémoire familiale 1-50, Chunk: Pour la mémoire familiale 1-50_Chunk1, Total Chunks: 2
🔢 Extracted Chunk Number: 1, Total Chunks: 2
🔍 Before Casting - min_chunk: 1 (<class 'int'>), max_chunk: 2 (<class 'int'>)
✅ After Casting - min_chunk: 1 (<class 'int'>), max_chunk: 2 (<class 'int'>)
🧩 Expanding context with bounded chunks: ['Pour la mémoire familiale 1-50_Chunk1', 'Pour la mémoire familiale 1-50_Chunk2']
🔍 Debugging Chunk IDs: ['Pour la mémoire familiale 1-50_Chunk1', 'Pour la mémoire familiale 1-50_Chunk2', 'Pour la mémoire familiale 1-50_Chunk3', 'Pour la mémoire familiale 1-50_Chunk4', 'Pour la mémoire familiale 1-50_Chunk5', 'Pour la mémoire familiale 1-50_Chunk6', 'Pour la mémoire familiale 1-50_Chunk7', 'Pour la mémoire familiale 1-50_Chunk8']
🔍 Debugging Chunk IDs: ['Pour la mémoire familiale 1-50_Chunk1', 'Pour la mémoire familiale 1-50_Chunk2', 'Pour la mémoire familiale 1-50_Chunk3', 'Pour la mémoire familiale 1-50_Chunk4', 'Pour la mémoire famil

'📜 Found Chunk: Pour la mémoire familiale 1-50_Chunk1 -> Dans le moment de régénération où les abus sont détruits, où les préjugés disparaissent, et quand l\'Assemblée Nationale a déclaré dans ses décrets augustes que les hommes naissent et demeurent égaux.\nNe sera-t-il pas permis aux juifs établis à Sarreguemines, de réclamer cette égalité si précieuse et si juste?.....\nDans l\'ivresse universelle qu\'inspirent des décisions à jamais mémorables, faudra-t-il que les juifs soient condamnés à gémir sur leur sort?" (Cité par Camille Wolff dans "L\'alliance Israelite" LVIII\'année-n* 17- 29/4/1897)\nCette pétition a été lue à l\'Assemblée le 3 août 1798, mais n\'a pas eu de suite.\nEntre temps, le 20 juillet, l\'Assemblée a décidé la suppression des taxes levées sur les juifs qui ont été maintenues malgré l\'abolition des privilèges. C\'est le cas entre autres de la lourde taxe levée en Lorraine en faveur du duc de Brancas. Mais les 12 août et 11 septembre de la même année, l\'Assemblée,

In [9]:
chunk_search.run({"query": "Qui est Francis?", "chunk_id": "Pour la mémoire familiale 1-50_Chunk4"})


🔍 Debugging Chunk IDs: ['Pour la mémoire familiale 1-50_Chunk1', 'Pour la mémoire familiale 1-50_Chunk2', 'Pour la mémoire familiale 1-50_Chunk3', 'Pour la mémoire familiale 1-50_Chunk4', 'Pour la mémoire familiale 1-50_Chunk5', 'Pour la mémoire familiale 1-50_Chunk6', 'Pour la mémoire familiale 1-50_Chunk7', 'Pour la mémoire familiale 1-50_Chunk8', 'Pour la mémoire familiale 1-50_Chunk9', 'Pour la mémoire familiale 1-50_Chunk10', 'Pour la mémoire familiale 1-50_Chunk11']


'📜 Found Chunk: Pour la mémoire familiale 1-50_Chunk2 -> siècle la position des philosophes du "siècle des lumières" n\'est pas très\nfavorable. Si les idées de respect de la personne humaine sont positives, par contre la tendance à\nl\'athéisme les conduit à rejeter le judaisme comme étant la racine du christianisme. Voltaire publie des\nécrits très virulents et de mauvaise foi contre les juifs. Rousseau leur serait plutôt favorable et\nMontesquieu choisit de les ignorer. Globalement les philosophes français s\'en préoccupent fort peu.\n\n-Les communautés juives en France.\n\nNous avons vu qu\'au début du XVI<sup>e</sup> siècle il n\'y a officiellement plus de juif en France. A\nla fin du XVIII<sup>e</sup> siècle, le nombre de juifs dont la présence est reconnue par le pouvoir royal est de\nl\'ordre de 40.000. Nous verrons plus loin les conditions de cette évolution....\n📜 Found Chunk: Pour la mémoire familiale 1-50_Chunk2 -> a quelques juifs, fournisseurs des armées ou\nbanquiers, qu

In [10]:
from langchain_core.messages import SystemMessage, AIMessage

@tool("report")
def report(
    introduction: str,
    research_steps: str,
    main_body: str,
    conclusion: str,
    sources: Union[str, List[str], None] = None
):
    """
    You are the family safe, keeper of the family collective memory.

    Returns a natural language response to the user's question based on the family memory database.

    Do not invent anything but you are allowed to link information together from multiple sources.
    """

    # Format the research steps and sources
    if isinstance(research_steps, list):
        research_steps = "\n".join([f"- {r}" for r in research_steps])
    if isinstance(sources, list):
        sources = "\n".join([f"- {s}" for s in sources])

    # System prompt to guide the final response
    system_prompt = SystemMessage(
        content=(
            "You are the family safe, the guardian of collective memory. "
            "Provide a detailed but concise summary based on the following sections: "
            "introduction, research steps, main body, and conclusion. "
            "Include sources if available and only use verified information."
            "Always make sure that conext was checked ussing **chunk_search**"
            "If you can't find an answer with sources, just say that you don't know."
        )
    )

    # Combine the sections into a message
    user_query_context = (
        f"### Introduction:\n{introduction}\n\n"
        f"### Research Steps:\n{research_steps}\n\n"
        f"### Main Body:\n{main_body}\n\n"
        f"### Conclusion:\n{conclusion}\n\n"
        f"### Sources:\n{sources if sources else 'No sources available.'}"
    )

    # Append the system prompt to messages and call the LLM
    messages = [system_prompt, AIMessage(content=user_query_context)]
    final_response = llm.invoke(messages)  # Call the LLM for a response

    return final_response


## Final answer Template

In [11]:
from langchain_core.messages import SystemMessage, AIMessage

@tool("final_answer")
def final_answer(
    answer: str,
    explore_next: str,
    sources: Union[str, List[str], None] = None
):
    """
    You are the family safe, keeper of the family collective memory.

    Returns a natural language response to the user's question based on the family memory database.

    Do not invent anything but you are allowed to link information together from multiple sources.

    If you can't find an answer, just say that you don't know.
    """

    # Format the research steps and sources
    if isinstance(sources, list):
        sources = "\n".join([f"- {s}" for s in sources])

    # System prompt to guide the final response
    system_prompt = SystemMessage(
        content=(
            "You are the family safe, the guardian of collective memory. "
            "Provide a detailed but concise answer to the original query."
            "If a report is requested, use {'tool': 'report'} to find more information."
            "Include sources if available and only use verified information."
            "If you can't find an answer with sources just say that you don't know."
            "Always make sure that context was checked using {'tool': 'chunk_search'}"
            "Offer to explore more information with the 'explore_next' prompt."
            "Final answer must always be in English."
        )
    )

    # Combine the sections into a message
    user_query_context = (
        f"### Answer:\n{answer}\n\n"
        f"### Sources:\n{sources if sources else 'No sources available.'}"
        f"### Explore Next:\n{explore_next}"

    )

    # Append the system prompt to messages and call the LLM
    messages = [system_prompt, AIMessage(content=user_query_context)]
    final_response = llm.invoke(messages)  # Call the LLM for a response

    return final_response


## Initialize Oracle (aka. Family keeper)

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

system_prompt = """You are the Family Safe, keeper of the family's collective memory. 
Your role is to decide how to handle user queries using the available tools.

**Tool Usage:**
- Do NOT reuse a tool for the same query (check the scratchpad).
- Do NOT use any tool more than **3 times**.
- Prioritize **rag_search** for gathering information.
- Use **chunk_search** to find context around a specific chunk.
- Do not mix sources from different contexts unless necessary.
- Alsways check at leats 2 ***Doc name*** to ensure the information is correct.

**Response Protocol:**
- If tools provides no answer, state that you don't know or can't any information about this topic"
- NEVER invent information or use data beyond the family memory.
- Always provide sources via the **final_answer** tool.
- Chunk_search must be in the scracthpad to point to final_answer.
- Discard any page content that looks like a table of content: you won't find any useful information there apart from page numbers.

By following these rules, you ensure accurate and responsible responses."""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("assistant", "scratchpad: {scratchpad}"),
])

## Initialize Agent

In [13]:
from langchain_core.messages import ToolCall, ToolMessage
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    openai_api_key = OPENAI_API_KEY,
    temperature=0
)

tools=[
    #simple_search,
    chunk_search,
    rag_search,
    report,
    # translate_to_french,
    final_answer
]

# define a function to transform intermediate_steps from list
# of AgentAction to scratchpad string
def create_scratchpad(intermediate_steps: list[AgentAction]):
    research_steps = []
    for i, action in enumerate(intermediate_steps):
        if action.log != "TBD":
            # this was the ToolExecution
            research_steps.append(
                f"Tool: {action.tool}, input: {action.tool_input}\n"
                f"Output: {action.log}"
            )
    return "\n---\n".join(research_steps)

oracle = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "scratchpad": lambda x: create_scratchpad(
            intermediate_steps=x["intermediate_steps"]
        ),
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="any")
)

### Test agent

In [14]:
inputs = {
    "input": "tell me something interesting about the Dreyfus family",
    "chat_history": [],
    "intermediate_steps": [],
}
out = oracle.invoke(inputs)
out

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_FrqehTvX1UQWcXv8zRwc1iLe', 'function': {'arguments': '{"query":"Dreyfus family"}', 'name': 'rag_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 508, 'total_tokens': 527, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-fea1ec04-4011-4910-94cf-8c3277991b81-0', tool_calls=[{'name': 'rag_search', 'args': {'query': 'Dreyfus family'}, 'id': 'call_FrqehTvX1UQWcXv8zRwc1iLe', 'type': 'tool_call'}], usage_metadata={'input_tokens': 508, 'output_tokens': 19, 'total_tokens': 527, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio

## Define Nodes for Graph

In [15]:
def run_oracle(state: list):
    print("run_oracle")
    print(f"intermediate_steps: {state['intermediate_steps']}")
    out = oracle.invoke(state)
    tool_name = out.tool_calls[0]["name"]
    tool_args = out.tool_calls[0]["args"]
    action_out = AgentAction(
        tool=tool_name,
        tool_input=tool_args,
        log="TBD"
    )
    return {
        "intermediate_steps": [action_out]
    }

def router (state: list): #Original
    # return the tool name to use
    if isinstance(state["intermediate_steps"], list):
        return state["intermediate_steps"][-1].tool
    else:
        # if we output bad format go to final answer
        print("Router invalid format")
#         return "final_answer"

In [16]:
tool_str_to_func = {
    "rag_search": rag_search,
    "report": report,
    #"simple_search": simple_search,
    "translate_to_french": translate_to_french,
    "chunk_search": chunk_search,
    "final_answer": final_answer
}

def run_tool(state: list):
    # use this as helper function so we repeat less code
    tool_name = state["intermediate_steps"][-1].tool
    tool_args = state["intermediate_steps"][-1].tool_input
    print(f"{tool_name}.invoke(input={tool_args})")
    # run tool
    out = tool_str_to_func[tool_name].invoke(input=tool_args)
    action_out = AgentAction(
        tool=tool_name,
        tool_input=tool_args,
        log=str(out)
    )
    return {"intermediate_steps": [action_out]}

## Define graph

In [17]:
from langgraph.graph import StateGraph, END

graph = StateGraph(AgentState)

graph.add_node("oracle", run_oracle)
#graph.add_node("simple_search", run_tool)
# graph.add_node("translate_to_french", run_tool)
graph.add_node("rag_search", run_tool)
graph.add_node("chunk_search", run_tool)
graph.add_node("report", run_tool)
graph.add_node("final_answer", run_tool)

graph.set_entry_point("oracle")

graph.add_conditional_edges(
    source="oracle",  # where in graph to start
    path=router,  # function to determine which node is called
)

# create edges from each tool back to the oracle
for tool_obj in tools:
    if tool_obj.name != "final_answer":
        graph.add_edge(tool_obj.name, "oracle")

# if anything goes to final answer, it must then move to END
graph.add_edge("final_answer", END)

runnable = graph.compile()

## Test agent

In [29]:
out = runnable.invoke({
    "input": "Qui est Jacques Drefyus?",
    "chat_history": [],
})

run_oracle
intermediate_steps: []
rag_search.invoke(input={'query': 'Jacques Drefyus'})
🔍 Best Match - Doc: Pdf img, Chunk: Pdf img_Chunk2, Total Chunks: 2
🔢 Extracted Chunk Number: 2, Total Chunks: 2
🔍 Before Casting - min_chunk: 1 (<class 'int'>), max_chunk: 2 (<class 'int'>)
✅ After Casting - min_chunk: 1 (<class 'int'>), max_chunk: 2 (<class 'int'>)
🧩 Expanding context with bounded chunks: ['Pdf img_Chunk1', 'Pdf img_Chunk2']
🔍 Debugging Chunk IDs: ['Pdf img_Chunk1', 'Pdf img_Chunk2', 'Pdf img_Chunk3', 'Pdf img_Chunk4', 'Pdf img_Chunk5', 'Pdf img_Chunk6', 'Pdf img_Chunk7', 'Pdf img_Chunk8']
🔍 Debugging Chunk IDs: ['Pdf img_Chunk1', 'Pdf img_Chunk2', 'Pdf img_Chunk3', 'Pdf img_Chunk4', 'Pdf img_Chunk5', 'Pdf img_Chunk6', 'Pdf img_Chunk7', 'Pdf img_Chunk8', 'Pdf img_Chunk9']
run_oracle
intermediate_steps: [AgentAction(tool='rag_search', tool_input={'query': 'Jacques Drefyus'}, log='TBD'), AgentAction(tool='rag_search', tool_input={'query': 'Jacques Drefyus'}, log="📜 Found Chunk: Pdf 

In [19]:
out = runnable.invoke({
    "input": "Who is Jacques Drefyus?",
    "chat_history": [],
})

run_oracle
intermediate_steps: []
rag_search.invoke(input={'query': 'Jacques Drefyus'})
🔍 Best Match - Doc: Pdf img, Chunk: Pdf img_Chunk2, Total Chunks: 2
🔢 Extracted Chunk Number: 2, Total Chunks: 2
🔍 Before Casting - min_chunk: 1 (<class 'int'>), max_chunk: 2 (<class 'int'>)
✅ After Casting - min_chunk: 1 (<class 'int'>), max_chunk: 2 (<class 'int'>)
🧩 Expanding context with bounded chunks: ['Pdf img_Chunk1', 'Pdf img_Chunk2']
🔍 Debugging Chunk IDs: ['Pdf img_Chunk1', 'Pdf img_Chunk2', 'Pdf img_Chunk3', 'Pdf img_Chunk4', 'Pdf img_Chunk5', 'Pdf img_Chunk6', 'Pdf img_Chunk7', 'Pdf img_Chunk8']
🔍 Debugging Chunk IDs: ['Pdf img_Chunk1', 'Pdf img_Chunk2', 'Pdf img_Chunk3', 'Pdf img_Chunk4', 'Pdf img_Chunk5', 'Pdf img_Chunk6', 'Pdf img_Chunk7', 'Pdf img_Chunk8', 'Pdf img_Chunk9']
run_oracle
intermediate_steps: [AgentAction(tool='rag_search', tool_input={'query': 'Jacques Drefyus'}, log='TBD'), AgentAction(tool='rag_search', tool_input={'query': 'Jacques Drefyus'}, log="📜 Found Chunk: Pdf 

In [20]:
out = runnable.invoke({
    "input": "Do we have info about La Belle au bois dormant in the family database?",
    "chat_history": []
})

run_oracle
intermediate_steps: []
rag_search.invoke(input={'query': 'La Belle au bois dormant'})
🔍 Best Match - Doc: Pour la mémoire familiale 1-50, Chunk: Pour la mémoire familiale 1-50_Chunk1, Total Chunks: 1
🔢 Extracted Chunk Number: 1, Total Chunks: 1
🔍 Before Casting - min_chunk: 1 (<class 'int'>), max_chunk: 1 (<class 'int'>)
✅ After Casting - min_chunk: 1 (<class 'int'>), max_chunk: 1 (<class 'int'>)
🧩 Expanding context with bounded chunks: ['Pour la mémoire familiale 1-50_Chunk1']
🔍 Debugging Chunk IDs: ['Pour la mémoire familiale 1-50_Chunk1', 'Pour la mémoire familiale 1-50_Chunk2', 'Pour la mémoire familiale 1-50_Chunk3', 'Pour la mémoire familiale 1-50_Chunk4', 'Pour la mémoire familiale 1-50_Chunk5', 'Pour la mémoire familiale 1-50_Chunk6', 'Pour la mémoire familiale 1-50_Chunk7', 'Pour la mémoire familiale 1-50_Chunk8']
run_oracle
intermediate_steps: [AgentAction(tool='rag_search', tool_input={'query': 'La Belle au bois dormant'}, log='TBD'), AgentAction(tool='rag_search'

In [21]:
out = runnable.invoke({
    "input": "Qui est Francis?",
    "chat_history": []
})

run_oracle
intermediate_steps: []
rag_search.invoke(input={'query': 'Francis'})
🔍 Best Match - Doc: Pour la mémoire familiale 1-50, Chunk: Pour la mémoire familiale 1-50_Chunk2, Total Chunks: 2
🔢 Extracted Chunk Number: 2, Total Chunks: 2
🔍 Before Casting - min_chunk: 1 (<class 'int'>), max_chunk: 2 (<class 'int'>)
✅ After Casting - min_chunk: 1 (<class 'int'>), max_chunk: 2 (<class 'int'>)
🧩 Expanding context with bounded chunks: ['Pour la mémoire familiale 1-50_Chunk1', 'Pour la mémoire familiale 1-50_Chunk2']
🔍 Debugging Chunk IDs: ['Pour la mémoire familiale 1-50_Chunk1', 'Pour la mémoire familiale 1-50_Chunk2', 'Pour la mémoire familiale 1-50_Chunk3', 'Pour la mémoire familiale 1-50_Chunk4', 'Pour la mémoire familiale 1-50_Chunk5', 'Pour la mémoire familiale 1-50_Chunk6', 'Pour la mémoire familiale 1-50_Chunk7', 'Pour la mémoire familiale 1-50_Chunk8']
🔍 Debugging Chunk IDs: ['Pour la mémoire familiale 1-50_Chunk1', 'Pour la mémoire familiale 1-50_Chunk2', 'Pour la mémoire familia

In [22]:
out = runnable.invoke({
    "input": "Qui sont les 4 frères de Francis?",
    "chat_history": [],
})

run_oracle
intermediate_steps: []
rag_search.invoke(input={'query': 'frères de Francis'})
🔍 Best Match - Doc: Pour la mémoire familiale 1-50, Chunk: Pour la mémoire familiale 1-50_Chunk2, Total Chunks: 2
🔢 Extracted Chunk Number: 2, Total Chunks: 2
🔍 Before Casting - min_chunk: 1 (<class 'int'>), max_chunk: 2 (<class 'int'>)
✅ After Casting - min_chunk: 1 (<class 'int'>), max_chunk: 2 (<class 'int'>)
🧩 Expanding context with bounded chunks: ['Pour la mémoire familiale 1-50_Chunk1', 'Pour la mémoire familiale 1-50_Chunk2']
🔍 Debugging Chunk IDs: ['Pour la mémoire familiale 1-50_Chunk1', 'Pour la mémoire familiale 1-50_Chunk2', 'Pour la mémoire familiale 1-50_Chunk3', 'Pour la mémoire familiale 1-50_Chunk4', 'Pour la mémoire familiale 1-50_Chunk5', 'Pour la mémoire familiale 1-50_Chunk6', 'Pour la mémoire familiale 1-50_Chunk7', 'Pour la mémoire familiale 1-50_Chunk8']
🔍 Debugging Chunk IDs: ['Pour la mémoire familiale 1-50_Chunk1', 'Pour la mémoire familiale 1-50_Chunk2', 'Pour la mémoi

In [30]:
out = runnable.invoke({
    "input": "A quelle date est morte la mère de Jean Lambert ?",
    "chat_history": []
})

run_oracle
intermediate_steps: []
rag_search.invoke(input={'query': 'mère de Jean Lambert date de décès'})
🔍 Best Match - Doc: Pour la mémoire familiale 1-50, Chunk: Pour la mémoire familiale 1-50_Chunk1, Total Chunks: 1
🔢 Extracted Chunk Number: 1, Total Chunks: 1
🔍 Before Casting - min_chunk: 1 (<class 'int'>), max_chunk: 1 (<class 'int'>)
✅ After Casting - min_chunk: 1 (<class 'int'>), max_chunk: 1 (<class 'int'>)
🧩 Expanding context with bounded chunks: ['Pour la mémoire familiale 1-50_Chunk1']
🔍 Debugging Chunk IDs: ['Pour la mémoire familiale 1-50_Chunk1', 'Pour la mémoire familiale 1-50_Chunk2', 'Pour la mémoire familiale 1-50_Chunk3', 'Pour la mémoire familiale 1-50_Chunk4', 'Pour la mémoire familiale 1-50_Chunk5', 'Pour la mémoire familiale 1-50_Chunk6', 'Pour la mémoire familiale 1-50_Chunk7', 'Pour la mémoire familiale 1-50_Chunk8']
run_oracle
intermediate_steps: [AgentAction(tool='rag_search', tool_input={'query': 'mère de Jean Lambert date de décès'}, log='TBD'), AgentActi

In [31]:
out = runnable.invoke({
    "input": "Qui est mort le 29 mars 1963?",
    "chat_history": []
})

run_oracle
intermediate_steps: []
rag_search.invoke(input={'query': 'mort le 29 mars 1963'})
🔍 Best Match - Doc: Pour la mémoire familiale 1-50, Chunk: Pour la mémoire familiale 1-50_Chunk1, Total Chunks: 2
🔢 Extracted Chunk Number: 1, Total Chunks: 2
🔍 Before Casting - min_chunk: 1 (<class 'int'>), max_chunk: 2 (<class 'int'>)
✅ After Casting - min_chunk: 1 (<class 'int'>), max_chunk: 2 (<class 'int'>)
🧩 Expanding context with bounded chunks: ['Pour la mémoire familiale 1-50_Chunk1', 'Pour la mémoire familiale 1-50_Chunk2']
🔍 Debugging Chunk IDs: ['Pour la mémoire familiale 1-50_Chunk1', 'Pour la mémoire familiale 1-50_Chunk2', 'Pour la mémoire familiale 1-50_Chunk3', 'Pour la mémoire familiale 1-50_Chunk4', 'Pour la mémoire familiale 1-50_Chunk5', 'Pour la mémoire familiale 1-50_Chunk6', 'Pour la mémoire familiale 1-50_Chunk7', 'Pour la mémoire familiale 1-50_Chunk8']
🔍 Debugging Chunk IDs: ['Pour la mémoire familiale 1-50_Chunk1', 'Pour la mémoire familiale 1-50_Chunk2', 'Pour la mé

In [25]:
out = runnable.invoke({
    "input": "Hello",
    "chat_history": []
})

run_oracle
intermediate_steps: []
final_answer.invoke(input={'answer': 'Hello! How can I assist you today?', 'explore_next': 'Feel free to ask any questions you have about family history or memories.', 'sources': None})


In [26]:
out = runnable.invoke({
    "input": "Fill me in on the intro of Pour la mémoire familiale 1-50",
    "chat_history": []
})

run_oracle
intermediate_steps: []
rag_search.invoke(input={'query': 'Pour la mémoire familiale 1-50 introduction'})
🔍 Best Match - Doc: Pour la mémoire familiale 1-50, Chunk: Pour la mémoire familiale 1-50_Chunk1, Total Chunks: 1
🔢 Extracted Chunk Number: 1, Total Chunks: 1
🔍 Before Casting - min_chunk: 1 (<class 'int'>), max_chunk: 1 (<class 'int'>)
✅ After Casting - min_chunk: 1 (<class 'int'>), max_chunk: 1 (<class 'int'>)
🧩 Expanding context with bounded chunks: ['Pour la mémoire familiale 1-50_Chunk1']
🔍 Debugging Chunk IDs: ['Pour la mémoire familiale 1-50_Chunk1', 'Pour la mémoire familiale 1-50_Chunk2', 'Pour la mémoire familiale 1-50_Chunk3', 'Pour la mémoire familiale 1-50_Chunk4', 'Pour la mémoire familiale 1-50_Chunk5', 'Pour la mémoire familiale 1-50_Chunk6', 'Pour la mémoire familiale 1-50_Chunk7', 'Pour la mémoire familiale 1-50_Chunk8']
run_oracle
intermediate_steps: [AgentAction(tool='rag_search', tool_input={'query': 'Pour la mémoire familiale 1-50 introduction'}, log

## Gradio

In [27]:
# Install necessary dependencies if needed
# %pip install --upgrade gradio langchain langgraph openai

import gradio as gr
import re
import ast

def extract_and_format_final_answer(data):
    """
    Extracts the latest `final_answer` from intermediate_steps in the given data,
    then formats and returns it in a structured output.

    Args:
        data (dict): The dictionary containing the 'intermediate_steps' list.

    Returns:
        str: A formatted string containing <answer> and <explore next>, and optionally <source>.
    """
    # Extract only the steps where tool == 'final_answer'
    final_answers = [step for step in data.get("intermediate_steps", []) if step.tool == "final_answer"]

    # Get the latest final answer and convert to string
    if not final_answers:
        return "⚠ No final answer found."
    
    log_string = str(final_answers[-1])

    # Extracting the tool_input dictionary using regex
    match = re.search(r"tool_input=({.*?})", log_string)
    if not match:
        return "⚠ No valid final answer found."

    # Convert the extracted string to a dictionary
    try:
        tool_input = ast.literal_eval(match.group(1))  # Safely convert string to dictionary
    except (SyntaxError, ValueError):
        return "⚠ Error parsing tool input."

    # Extract required fields
    answer = tool_input.get('answer', 'No answer provided.')
    sources = tool_input.get('sources', None)  # Keep None instead of default text
    explore_next = tool_input.get('explore_next', 'No further suggestions.')

    # Construct the formatted output
    formatted_output = f"{answer}\n"

    # Append source only if it's not None or "No sources available"
    if sources and sources != "No sources available":
        formatted_output += f"\nsource : {sources}\n"

    # Append explore next
    formatted_output += f"\n{explore_next}"

    return formatted_output.strip()




def query_oracle(text: str):
    """Invokes the LangGraph agent and extracts a cleaned response."""
    response = runnable.invoke({
        "input": text,
        "chat_history": []
    })
    # return extract_final_answer(response)
    return extract_and_format_final_answer(response)
        
    
    # except Exception as e:
    #     return f"Error: {str(e)}"

# Set up Gradio interface
interface = gr.Interface(
    fn=query_oracle,
    inputs=gr.Textbox(label="Enter your query"),
    outputs=gr.Textbox(label="Response"),
    title="LangGraph Assistant",
    description="Ask a question and get a structured response with sources and next steps."
)

# Launch Gradio app
interface.launch()


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
